In [ ]:
# install 
!pip install -e https://github.com/eltoto1219/vltk.git

In [ ]:
from vltk.abc.extraction import VizExtractionAdapter, VizExtractionAdapters
from vltk.abc.visnadapter import VisnDatasetAdatpter, VisnDatasetAdapters
from vltk.abc.visnlangadatper import VisnLangDatasetAdapter, VisnLangDatasetAdapters
import vltk
from vltk.modeling.frcnn import FRCNN as FasterFRCNN
from vltk.configs import ProcessorConfig
from vtk import compat
import os

In [ ]:
TESTDIR = "/home/eltoto/vltk/tests"
DATADIR = "/home/eltoto/data"
COCODIR = "/home/eltoto/vltk/tests/coco"
VGDIR =   "/home/eltoto/vltk/tests/visualgenome"

In [ ]:
# define visual feature extraction adapter for FRCNN
class FRCNN(VizExtractionAdapter):

        default_processor = ProcessorConfig(
        **{
            "transforms": ["ToPILImage", "ToTensor", "ResizeTensor", "Normalize"],
            "size": (800, 1333),
            "mode": "bilinear",
            "pad_value": 0.0,
            "mean": [102.9801, 115.9465, 122.7717],
            "sdev": [1.0, 1.0, 1.0]
        }
    )
    model_config = compat.Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")
    weights = "unc-nlp/frcnn-vg-finetuned"
    model = FasterRCNN
    ...

In [ ]:
# define vision dataset adatpers for COCO + visualo genome
class Coco2014(VisnDatasetAdapter):
    def schema():
        return {
          vltk.box: Features.box
          vltk.segmentation: Features.segmentation
        }
    ...
    
class VisualGenome(VisnDatasetAdapter):
    def schema():
        return {
          vltk.box: Features.box
          vltk.segmentation: Features.segmentation
        }
    ...

In [ ]:
# define vision-language dataset adapters for VQA + GQA
 7 class GQA(VisnLangDatasetAdapter):
  6     name = "gqa"
  5     data_info = {
  4     ¦   "dev": {"coco2014": ["test"]},
  3     ¦   "train": {"vg": ["train"]},
  2     ¦   "val": {"vg": ["train"]},
  1     ¦   "test": {"coco2014": ["test"]},
17      }
  1     default_features = {
  2     ¦   "qid": ds.Value("string"),
  3     ¦   "structure": ds.Value("string"),
  4     ¦   "super_class": ds.Value("string"),
  5     ¦   "operations": ds.Sequence(length=-1, feature=ds.Value("string")),
  6     }
class VQA(VisnLangDatasetAdapter):
 14     name = "vqa"
 15     data_info = {
 16     ¦   "val": {"coco2014": ["val"]},
 17     ¦   "train": {"coco2014": ["train"]},
 18     ¦   "test": {"coco2014": ["test"]},
 19     }
 20     default_features = {
 21     ¦   "qid": ds.Value("string"),
 22     }
 23

In [ ]:
# add adapters to library
VisnLangDatasetAdapters.add("vqa", VQA)
VisnLangDatasetAdapters.add("gqa", GQA)
VisnDatasetAdapters.add("coco2014", Coco2014)
VisnDatasetAdapters.add("visualgenome", VisualGenome)
VizExtractionAdapters.add("frcnn", FRCNN)

In [ ]:
# superset datasets together

from vltk import DataConfigimport vltk
#define config for datasetconfig = DataConfig(
# choose which dataset and dataset split for train and eval
train_datasets=[["gqa’, ’train’], [’vqa’,’trainval’]],
eval_datasetts=[’gqa’, ’testdev’],
# choose which tokenizer to use
tokenizer=’BertWordPeice’,# choose which feature extractor to use
extractor=’frcnn’)
# use config to create dataset
dataset = vltk.init_dataset(config)
# first entry in the dataset
dataset[0]:

In [ ]:
from vltk import VLDataset
# iterate through dataset
for batch in dataset:
    print(batch)
    VLDataset.transpose(batch)
    print(batch)
    break
    